In [49]:
from __future__ import division
import pandas as pd
import numpy as np
import os

# Settings

In [80]:
n_runs = 3
n_sessions = 2
n_trials = 120
tr = 3
jitters = [0.75, 1.5, 2.25, 3]
n_stop_trials = int(np.ceil(n_trials/4))
perc = (n_stop_trials / n_trials)*100

print('Settings:\n\n\
Sessions: {n_sessions}\n\
Runs per session: {n_runs}\n\
Trials per run: {n_trials}\n\
Stop trials per run: {n_stop_trials} ({perc:.2f} percent)\n\
Assuming a TR of {tr} seconds\n\
Jitter options: {jitters} seconds'.format(**globals()))

Settings:

Sessions: 2
Runs per session: 3
Trials per run: 120
Stop trials per run: 30 (25.00 percent)
Assuming a TR of 3 seconds
Jitter options: [0.75, 1.5, 2.25, 3] seconds


In [81]:
def generate_design(n_trials, jitters, n_stop_trials):
    # make direction array
    
    design = pd.DataFrame({'direction': np.tile([0, 1], int(np.ceil(n_trials/2))),
                           'stop_trial': 0})
    design.loc[:(n_stop_trials-1), 'stop_trial'] = 1
    design = design.sample(frac=1).reset_index(drop=True)
    
    # make jitter array
    design['jitter'] = np.random.choice(jitters, size=n_trials, replace=True)

    return(design)

In [82]:
n_subjects = 18

In [84]:
#
save_dir = './designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.arange(1,n_subjects+1):
    for session in range(1,n_sessions+1):
        designs_this_session = []
        
        for run in range(1,n_runs+1):
            design = generate_design(n_trials, jitters, n_stop_trials)
            design['block'] = run
            
            designs_this_session.append(design)
        
        design = pd.concat(designs_this_session)
        fn = 'sub-' + str(subject_id).zfill(3) + '_session-' + str(session) + '_design'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

sub-001_session-1_design
sub-001_session-2_design
sub-002_session-1_design
sub-002_session-2_design
sub-003_session-1_design
sub-003_session-2_design
sub-004_session-1_design
sub-004_session-2_design
sub-005_session-1_design
sub-005_session-2_design
sub-006_session-1_design
sub-006_session-2_design
sub-007_session-1_design
sub-007_session-2_design
sub-008_session-1_design
sub-008_session-2_design
sub-009_session-1_design
sub-009_session-2_design
sub-010_session-1_design
sub-010_session-2_design
sub-011_session-1_design
sub-011_session-2_design
sub-012_session-1_design
sub-012_session-2_design
sub-013_session-1_design
sub-013_session-2_design
sub-014_session-1_design
sub-014_session-2_design
sub-015_session-1_design
sub-015_session-2_design
sub-016_session-1_design
sub-016_session-2_design
sub-017_session-1_design
sub-017_session-2_design
sub-018_session-1_design
sub-018_session-2_design


In [85]:
# for debug
n_runs = 3
n_sessions = 2
n_trials = 10
tr = 3
jitters = [0.75, 1.5, 2.25, 3]
n_stop_trials = int(np.ceil(n_trials/4))

save_dir = './designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in ['DEBUG']:
    for session in range(1,n_sessions+1):
        designs_this_session = []
        
        for run in range(1,n_runs+1):
            design = generate_design(n_trials, jitters, n_stop_trials)
            design['block'] = run
            
            designs_this_session.append(design)
        
        design = pd.concat(designs_this_session)
        fn = 'sub-' + str(subject_id).zfill(3) + '_session-' + str(session) + '_design'
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

In [86]:
design.groupby('direction').stop_trial.sum()

direction
0    6
1    3
Name: stop_trial, dtype: int64